In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()
import os
import aspose.words as aw
import re
from guess_language import guess_language
import pytesseract
from pdf2image import convert_from_path

In [ ]:
from datasets import load_metric, Dataset
import datasets
import torch
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction

Подготовка текстов

In [13]:
labels = ['act', 'application', 'arrangement', 'bill', 'contract', 'contract offer', 'determination', 'invoice',
          'order', 'proxy', 'statute']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}


#для теста
def preprocess_test(examples):
    # take a batch of texts
    text = examples['text']
    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512) #max_length=128
  
    return encoding

def encoding_test(df_test):
    #подготовка датасета для модели
    #features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
    test_dataset = Dataset.from_pandas(df_test)
    dataset = datasets.DatasetDict({"test":test_dataset})
    
    encoded_dataset = dataset.map(preprocess_test, batched=True, remove_columns=dataset['test'].column_names)
    encoded_dataset.set_format("torch")
    
    return encoded_dataset

Загрузка моделей. Токенизатор от DeepPavlov. Модель из сохраненной папки

In [20]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

save_directory = 'training/results/checkpoint-5430'
best_model = BertForSequenceClassification.from_pretrained(save_directory, local_files_only=True)

trainer = Trainer(best_model)

Функции для предсказаний

In [4]:
def get_prediction(dataset):
    # предсказание для теста
    pred = trainer.predict(dataset)
    labels = np.argmax(pred.predictions, axis = -1)
    labels = [id2label[idx] for idx in list(labels)]
    return labels

def predict_text(text):
    #предсказание для одного текста
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().to("cpu"))
    predictions = np.zeros(probs.shape)
    #predictions[np.where(probs >= 0.5)] = 1  #мультилейбл
    predictions[np.where(probs == max(probs))] = 1  #один лейбл
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    return predicted_labels

Функции для считывания текстов

In [5]:
def read_train(file_list, path, pars="tika"):
    '''Считывание документов из папки для добавления в датафрейм.
    На входе:
       file_list - список файлов
       path - папка, в которой лежат файлы
       pars - парсер. Варианты: tika(все форматы), aspose.words(все), textract(все, особый способ для rtf),
                                docx(только docx), rtf(rtf), pdf(pdf).
           Проверка на корректность pdf, если текст не распознается как русский - распознавание pytesseract,
           если текст отсутсвет, по умолчанию "".
    На выходе:
       texts - список той же длины, что и file_list'''
    
    texts = []   #все тексты договоров
    for i in tqdm(file_list[0:]):
        filename = path+i
        if os.path.exists(filename):
            #проверку, чтобы docx тотолько в docx
            if pars=="tika":
                parsed = parser.from_file(filename)
                #text = parsed["content"]
                text = easy_clean(parsed["content"], parsed["metadata"])                
            elif pars=="aw":
                parsed = aw.Document(filename).get_text()
                text = aw_clean(parsed)
                #text = parsed
            if filename.split('.')[-1] == "pdf":
                #у pdf проверяем адекватность текста, если мусор - распознаем изображениями
                if guess_language(text) != "ru":
                    text = ''
                    pages = convert_from_path(filename, 500)
                    for i, page in enumerate(pages):
                        parsed = pytesseract.image_to_string(page, lang="rus")
                        text += parsed
                    text = easy_clean(text, [])
                else:
                    pass
                                                    
            texts.append(text)
        else:
            texts.append("")
            print(filename, ' не существует')
    return texts


def aw_clean(s):
    #простая очистка текста
    replace_dict={"Скачано с":" ", 'Образец документа':' ', 
                 'подготовлен сайтом\x13 HYPERLINK "https://dogovor-obrazets.ru"':' ',
                 '\x14https://dogovor-obrazets.ru\x15':' ',
                 'Источник страницы с документом:\x13 HYPERLINK "https://dogovor-obrazets.ru':' ',
                 '/Ð´Ð¾Ð³Ð¾Ð²Ð¾Ñ\x80/Ð\x9eÐ±Ñ\x80Ð°Ð·ÐµÑ\x86_Ð\x94Ð¾Ð³Ð¾Ð²Ð¾Ñ\x80_Ð¿Ð¾Ñ\x81Ñ':' ',
                  '\x82Ð°Ð²ÐºÐ¸_Ñ\x82Ð¾Ð²Ð°Ñ\x80Ð°-1" \x14https://dogovor-obrazets.ru/договор/':' ',
                  "\* MERGEFORMAT": " ", "FILLIN": " ",
                 'Evaluation Only. Created with Aspose.Words. Copyright 2003-2023 Aspose Pty Ltd.': " ",
                 'Created with an evaluation copy of Aspose.Words.': " ",
                 'To discover the full versions of our APIs please visit: https://products.aspose.com/words/':" ",
                 "This document was truncated here because it was created in the Evaluation Mode.": " ", 
                 }
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    s = re.sub(r"https?://[^,\s]+,?", " ", s) #удаление гиперссылок
    s = re.sub('"consultantplus://offline/ref=[0-9A-Z]*"', '', s)
    s = re.sub(r'HYPERLINK \\l "Par[0-9]*"', '', s)
    s = re.sub(r'HYPERLINK', '', s)
    #re.sub(r'\(<^\)>+\)', '', string) удалить <1>
    #s = re.sub(r"^\s+|\n|\t|\r|\s+$", " ", s) #удаление пробелов в начале и конце строки, переносов
    replace_dict={" .": ".", " ,": ",", " « »": "", "« » ": "", " « »": "",' " "': "", " “ ”": "",  
                  "_":" ", "�":" ", "·": "", "--":"", "…":"", "/":"", "|":"", '“”':'', "®": " ", "\d": " ", 
                  '\x0e':" ", "\x02":"", "\x0c":" ", "\x07":" ", "\xa0":" ", "\x13":" ", "\x14":" ", "\x15":" "} 
    for key, value in replace_dict.items():
        s = s.replace(key, value) 
    s = re.sub(r"\s+", " ", s).strip() #удаление пробелов в начале и конце строки, переносов \r\n\t
    replace_dict={" « »":"", " “ ”":"", "( ) ": ""}
    for key, value in replace_dict.items():
        s = s.replace(key, value) 
    #s = s.capitalize()
    return s

Функция для загрузки текста для приложения

In [6]:
def load_file(filename, pars="aw"):
    '''Считывание документов из загрузки.
    На входе:
       filename - название файла
       pars - парсер.aspose.words(все форматы)
           Проверка на корректность pdf, если текст не распознается как русский - распознавание pytesseract,
           если текст отсутсвет, по умолчанию "".
    На выходе:
       text - текст документа'''
    
    text = None
    #поддерживаемые типы файлов
    doc_type = ['docx', 'doc', 'rtf', 'pdf']
    
    #if file_path.name.rsplit('.', 1)[-1] in doc_type:
    if filename.split('.')[-1] in doc_type:
        parsed = aw.Document(filename).get_text()
        text = aw_clean(parsed)
        
    else:
        #неподдерживаемый тип
        pass

    if file_path.name.rsplit('.', 1)[-1] == 'pdf':
    #elif filename.split('.')[-1] == "pdf":
        #у pdf проверяем адекватность текста, если мусор - распознаем изображениями
        if guess_language(text) != "ru":
            text = ''
            pages = convert_from_path(filename, 500)
            for i, page in enumerate(pages):
                parsed = pytesseract.image_to_string(page, lang="rus")
                text += parsed
            text = easy_clean(text, [])
        else:
            pass


    return text

Маппинг класса к id теста:

proxy - 1  
contract - 2  
act - 3  
application - 4  
order - 5  
invoice - 6  
bill - 7  
arrangement - 8  
contract offer - 9  
statute - 10  
determination - 11  

Загрузка тестового датасета

In [16]:
test = pd.read_csv('dataset/dataset.csv', sep='|', index_col=None)
test.columns = ['document_id', 'text']
test.head()

,document_id,text
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...
3,-2949797178567945855,Приложение ...
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...


In [23]:
test.shape

(263, 2)

In [21]:
test_dataset = encoding_test(test[['text']])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
#предсказание
pred = get_prediction(test_dataset['test'])
test['class']=pred

Переводим как в сабмите

In [29]:
mapping = {'proxy' : 1, 'contract' : 2, 'act' : 3, 'application' : 4, 'order' : 5, 'invoice' : 6,
'bill' : 7, 'arrangement' : 8, 'contract offer' : 9, 'statute' : 10, 'determination' : 11}
test['class_id']=test['class'].map(mapping)
test

,document_id,text,class,class_id
0,-1704473130726551677,Приложение №1 к договору подряда № 2 от 25.07....,act,3
1,-8755312099564304902,Директору ООО «Вершки-Корешки» Головину Олегу ...,application,4
2,4407537979341024747,Приложение к договору ренты № 15 от 25.07.2024...,act,3
3,-2949797178567945855,Приложение ...,contract,2
4,-3027899216558317249,ООО «Электросталь»Приказ № 66/ВО выводе из экс...,order,5
...,...,...,...,...
258,3405154703345778066,ДоверенностьГ. Курган ...,proxy,1
259,4748799364308515784,ООО «Невада холдинг»Приказ № 123О назначении о...,order,5
260,8934602722747847927,Приложение № 2к Договору №____________ от «___...,contract,2
261,-6862048199031279271,Договор с покупателемг. Москва ...,contract,2


In [30]:
test[['document_id', 'class_id']].to_csv("subm1.csv", sep=';', index=False)

0.594122 на тесте. Модель слишком переобучилась на наших документах, необходимо увеличить датасет

Проверка загрузки отдельного файла документа

In [16]:
path = '/media/tanya/data/OLD/NLP_all/ZavodIt2022/docs/'
file=['7aff676ead9fe323b2c542e60accb1ae.doc']

In [17]:
texts = read_train(file, path='test/', pars="aw")

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
texts

['Приложение № ДОГОВОР № 20 г. (краткое и полное наименование), именуемое в дальнейшем «Заказчик», в лице , действующего на основании с одной стороны и (краткое и полное наименование), именуемoе в дальнейшем «Подрядчик», в лице , действующего на основании с другой стороны, при совместном упоминании в дальнейшем именуемые «Стороны», а по отдельности «Сторона», заключили настоящий договор подряда № от г. (далее – «Договор») о нижеследующем: 1. Предмет Договора Подрядчик обязуется по заданию Заказчика выполнить работу, предусмотренную настоящим Договором, а Заказчик обязуется принять ее результат и оплатить в соответствии с условиями Договора. Подрядчик обязуется выполнить следующие работы (далее по тексту – Работы): [Вариант 1: 1.2.1. 1.2.2. (указываются перечень, состав, результат Работ, которые Подрядчик должен выполнить) Вариант 2: Перечень и состав Работ, требования к результату Работ указаны в приложение № 1 к настоящему Договору, являющимся его неотъемлемой частью.] При большом объ

In [41]:
def predict_text(text):
    #предсказание для одного текста
    encoding = tokenizer(text, return_tensors="pt", max_length=512)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().to("cpu"))
    predictions = np.zeros(probs.shape)
    scores = max(probs).item()
    #predictions[np.where(probs >= 0.5)] = 1  #мультилейбл
    predictions[np.where(probs == max(probs))] = 1  #один лейбл
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    return predicted_labels[0], round(scores, 2)

In [42]:
predict_text(texts)

('contract', 0.96)